# Multiscale Analysis of the Sustainable Production of 1,3PDO and 3HP
## X2 - Sensitivity Analysis: Product Tolerance of the Cell Factories (Max Titer)

In [1]:
%reset -f
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Setting up Cell Factory Organisms with Product Tolerance level at 50%, 100%, and 150%

In [2]:
### Setting up in silico Organisms
import cPickle as pickle
from cbmTES.global_constants import *
from cbmTES.bioreactor import Ecoli, Scerevisiae

## Setting up E. coli
ecoli_model = pickle.load(open('models/ecoli_super.pickle','rb'))

# maximum titer of products are set to 50, 100, 150 g/L for E. coli
titer_limit = {}
titer_limit[r_13PDO] = 100.0/76*1000*0.5
titer_limit[r_3HPP] = 100.0/90*1000*0.5
ecoli_pt50 = Ecoli(ecoli_model, id='ecoli_pt50', titer_limit=titer_limit) # product tolerance at 50%

titer_limit = {}
titer_limit[r_13PDO] = 100.0/76*1000
titer_limit[r_3HPP] = 100.0/90*1000
ecoli_pt100 = Ecoli(ecoli_model, id='ecoli_pt100', titer_limit=titer_limit)

titer_limit = {}
titer_limit[r_13PDO] = 100.0/76*1000*1.5
titer_limit[r_3HPP] = 100.0/90*1000*1.5
ecoli_pt150 = Ecoli(ecoli_model, id='ecoli_pt150', titer_limit=titer_limit) # product tolerance at 50%



## Setting up S. cerevisiae 
scere_model = pickle.load(open('models/scere_super.pickle','rb'))

# maximum titer of products are set to 75, 150, 225 g/L for S. cerevisiae
titer_limit = {}
titer_limit[r_13PDO] = 150.0/76*1000*0.5
titer_limit[r_3HPP] = 150.0/90*1000*0.5
scere_pt50 = Scerevisiae(scere_model, id='scere_pt50', titer_limit=titer_limit)

titer_limit = {}
titer_limit[r_13PDO] = 150.0/76*1000
titer_limit[r_3HPP] = 150.0/90*1000
scere_pt100 = Scerevisiae(scere_model, id='scere_pt100', titer_limit=titer_limit)

titer_limit = {}
titer_limit[r_13PDO] = 150.0/76*1000*1.5
titer_limit[r_3HPP] = 150.0/90*1000*1.5
scere_pt150 = Scerevisiae(scere_model, id='scere_pt150', titer_limit=titer_limit)

Biomass reaction detected: R_Ec_biomass_iJO1366_core_53p95M
Biomass reaction detected: R_biomass_SC5_notrace


In [3]:
### Setting up a collection of in silico Bioreactors
from cbmTES.bioreactor import IdealFedbatch
from cbmTES.bioreactor import ecoli_parameters, scere_parameters

hosts = [ecoli_pt50, ecoli_pt100, ecoli_pt150, scere_pt50, scere_pt100, scere_pt150]
#hosts = [scere]
oxygen_availability = [AEROBIC, ANAEROBIC]
substrates = [r_glc, r_glyc]
product = [r_13PDO, r_3HPP]

# construct a bioreactor instance for each condition.
bioreactors = OrderedDict()

# host specific parameters
s_feed = {}
s_init = {}

for h in hosts:
    if 'ecoli' in h.id:
        s_feed[h.id, r_glc] = 500.0/180*1000 #Alves 2007
        s_init[h.id, r_glc] = 50.0/180*1000
        s_feed[h.id, r_glyc] = 500.0/92*1000
        s_init[h.id, r_glyc] = 100.0/92*1000
    if 'scere' in h.id:
        s_feed[h.id, r_glc] = 500.0/180*1000
        s_init[h.id, r_glc] = 200.0/180*1000
        s_feed[h.id, r_glyc] = 500.0/92*1000
        s_init[h.id, r_glyc] = 60.0/92*1000



conditions = [(h, o, s, p) for h in hosts for o in oxygen_availability for s in substrates for p in product if not (o == ANAEROBIC and s == r_glyc) ]

for condition in conditions:
    host = condition[0]
    oxygen_availability = condition[1]
    substrate = condition[2]
    product = condition[3]
    
    metabolites = [substrate, product]
    Sfeed = [s_feed[host.id, substrate], 0]
    bioreactors[host.id, oxygen_availability, substrate, product] = IdealFedbatch(metabolites=metabolites, Sfeed = Sfeed, volume_max=10, oxygen_availability=oxygen_availability)

    # initilize bioreactor
    Vinit = [1]
    Xinit = [0.01]
    Sinit = [s_init[host.id, substrate], 0]
    bioreactors[host.id,  oxygen_availability, substrate, product].set_initial_conditions(Vinit, Xinit, Sinit)
                

### Dynamic Scanning Analysis

In [4]:
from collections import OrderedDict
from time import time
import framed.bioreactor.dyssco as dyssco
import cPickle as pickle

# make 10 strains along production envelopes for each case
strains = OrderedDict()

for condition in conditions:
    host = condition[0]
    oxygen_availability = condition[1]
    substrate = condition[2]
    product = condition[3]
    print host.id, ':', oxygen_availability, ':', substrate, ':', product
    t0 = time()
    if 'ecoli' in host.id:
        strains[host.id, oxygen_availability, substrate, product] = dyssco.make_envelope_strains(host, substrate, product, N=10, constraints=fba_constraints['ecoli', substrate, oxygen_availability])
    elif 'scere' in host.id:
        strains[host.id, oxygen_availability, substrate, product] = dyssco.make_envelope_strains(host, substrate, product, N=10, constraints=fba_constraints['scere', substrate, oxygen_availability])
    t1 = time()
    print 'Simulation runtime: ', t1-t0
    
pickle.dump(strains,  open('jar/strains_sa_pt.pickle','wb'))

ecoli_pt50 : 1 : R_EX_glc_e : R_EX_13PDO_e
Biomass reaction detected: R_Ec_biomass_iJO1366_core_53p95M
Simulation runtime:  15.6226460934
ecoli_pt50 : 1 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  14.2670099735
ecoli_pt50 : 1 : R_EX_glyc_e : R_EX_13PDO_e
Simulation runtime:  15.5430569649
ecoli_pt50 : 1 : R_EX_glyc_e : R_EX_3HPP_e
Simulation runtime:  14.6087241173
ecoli_pt50 : 0 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  16.2215499878
ecoli_pt50 : 0 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  15.3172669411
ecoli_pt100 : 1 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  14.1819210052
ecoli_pt100 : 1 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  17.9359710217
ecoli_pt100 : 1 : R_EX_glyc_e : R_EX_13PDO_e
Simulation runtime:  15.777310133
ecoli_pt100 : 1 : R_EX_glyc_e : R_EX_3HPP_e
Simulation runtime:  14.5785579681
ecoli_pt100 : 0 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  15.9727990627
ecoli_pt100 : 0 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  15.0187859535
e

In [5]:
# simulate the metabolic activities of these strains using dynamic FBA
from collections import OrderedDict
from time import time
import framed.bioreactor.dyssco as dyssco
import cPickle as pickle

strains = pickle.load(open('jar/strains_sa_pt.pickle', 'rb'))

performances = OrderedDict()

dfba_solutions = {}

for condition in conditions:
    host = condition[0]
    oxygen_availability = condition[1]
    substrate = condition[2]
    product = condition[3]
    bioreactor=bioreactors[host.id, oxygen_availability, substrate, product]
    print host.id, ':', oxygen_availability, ':', substrate, ':', product
    t0 = time()
    strains_on_envelope = strains[host.id, oxygen_availability, substrate, product] # the 10 strains along the production envelop for a particular condition
    performance = dyssco.calculate_performances(strains_on_envelope, bioreactor, substrate, product, 0, 150, 0.2, verbose=False, additional_yields=['R_EX_co2_e', 'R_EX_nh4_e', 'R_EX_pi_e'], save_dfba_solution=True, dfba_solver='vode')
    t1 = time()
    print 'Simulation runtime: ', t1-t0
    performances[host.id, oxygen_availability, substrate, product] = performance
    
pickle.dump(performances,  open('jar/performances_sa_pt.pickle','wb'))

ecoli_pt50 : 1 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  70.9107811451
ecoli_pt50 : 1 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  34.7460479736
ecoli_pt50 : 1 : R_EX_glyc_e : R_EX_13PDO_e
Simulation runtime:  54.6665148735
ecoli_pt50 : 1 : R_EX_glyc_e : R_EX_3HPP_e
Simulation runtime:  35.1159110069
ecoli_pt50 : 0 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  42.2022798061
ecoli_pt50 : 0 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  61.8976020813
ecoli_pt100 : 1 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  74.3679068089
ecoli_pt100 : 1 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  45.5624811649
ecoli_pt100 : 1 : R_EX_glyc_e : R_EX_13PDO_e
Simulation runtime:  73.0670130253
ecoli_pt100 : 1 : R_EX_glyc_e : R_EX_3HPP_e
Simulation runtime:  65.8404550552
ecoli_pt100 : 0 : R_EX_glc_e : R_EX_13PDO_e
Simulation runtime:  57.6475701332
ecoli_pt100 : 0 : R_EX_glc_e : R_EX_3HPP_e
Simulation runtime:  82.1435220242
ecoli_pt150 : 1 : R_EX_glc_e : R_EX_13PDO_e
Simulation runti

/Users/kaizhuang/.virtualenvs/CfB/lib/python2.7/site-packages/framed/bioreactor/dyssco.py:216: RuntimeWarning: divide by zero encountered in double_scalars
  performance['productivity'] = performance['product_titer']/dfba_solution['time'][index]


In [6]:
# convert the performance metric from mol-based to gram-based 
import cPickle as pickle
performances = pickle.load(open('jar/performances_sa_pt.pickle', 'rb'))

def convert_mmol2gram(performance, substrate, product, chemical_mass_table):
    """
    Unit conversion: mmol -> g 
        for Bioprocess A->B, convert the base unit of strain performance metric from mmol to gram.
        
        Product Yield(s): mmol/mmol -> g/g (x molar_mass_B / molar_mass_A)
        Biomass Yield: g/mmol -> g/g (x 1000/molar_mass_A)
        Productivity: mmol/L/hr -> g/L/hr (x molar_mass_B/1000)
        Titer: mmol/L -> g/L (x molar_mass_B/1000)
    """    
    MM_A = chemical_mass_table[substrate]  # molar mass of the substrate
    MM_B = chemical_mass_table[product]  # molar mass of the product
    converter = {'product_yield': MM_B / MM_A,
                 'biomass_yield': 1000/MM_A,   
                 'productivity': MM_B/1000,
                 'product_titer': MM_B/1000         
                 }
    
    for c_id, c_factor in converter.items():
        performance[c_id] = performance[c_id]*c_factor
        
    
chemical_mass_table = {
r_glc:  180.16,  # g/mol
r_glyc: 92.09,   # g/mol
r_13PDO:  76.09,   # g/mol
r_3HPP:  90.08   # g/mol
}

for key, performance in performances.items():
    for p in performance:
        convert_mmol2gram(p, key[2], key[3], chemical_mass_table)
        
        
pickle.dump(performances,  open('jar/performances_sa_pt.pickle','wb'))

### Creating three Economy scale models with 50%, 100%, and 150% product tolerance

In [1]:
import cPickle as pickle
# from collections import OrderedDict
from cbmTES.global_constants import *
from framed.hack.ratio_constraint import *
import cbmTES.biochemical as biochemical
from framed.analysis.simulation import FBA, pFBA

performances = pickle.load(open('jar/performances_sa_pt.pickle', 'rb'))


# performances_pt50 = OrderedDict()
# performances_pt100 = OrderedDict()
# performances_pt150 = OrderedDict()

# for key in performances.keys():
#     if 'pt50' in key[0]:
#         performances_pt50[key] = performances[key]
#     elif 'pt100' in key[0]:
#         performances_pt100[key] = performances[key]
#     elif 'pt150' in key[0]:
#         performances_pt150[key] = performances[key]

In [2]:
# import the economy scale model
model = pickle.load(open('models/mini_BChI_model.pickle','rb'))

# remove the existing BioProcess Reactions 
for rxn in model.reactions:
    if 'BPR' in rxn:
        model.remove_reaction(rxn)
        
# Adding Bioprocesses to the model

for key in performances.keys():
    strains = performances[key]
    #strains = biochemical.get_strains_metric(metric)
    
    if key[1] == AEROBIC:
        bioreactor = 'aeroFB'
    elif key[1] == ANAEROBIC:
        bioreactor = 'anaeFB'
    
    if key[2] == r_glc:
        substrate = 'glc'
    elif key[2] == r_glyc:
        substrate = 'glyc'
        
    if key[3] == r_13PDO:
        product = '1,3pdo'
    if key[3] == r_3HPP:
        product = '3hp'

    for i, strain in enumerate(strains):
        if strain['product_yield']>0 and strain['biomass_yield']>0:
            if product == '3hp':
                biochemical.add_bioprocess_reaction(model, substrate, product, strain, bioreactor, 'ctrf', 'evapM', 1, neutralization='3hp_lime', process_energy_intensity=biochemical.Ei, compartment='bch')
            else:    
                biochemical.add_bioprocess_reaction(model, substrate, product, strain, bioreactor, 'ctrf', 'evapM', 1, process_energy_intensity=biochemical.Ei, compartment='bch')



### Sensitivity Analysis: Scenario 1A, B, C, D

In [3]:
## Setting up scenario 1A, B, C, D

### Objective Functions ###
OBJs=OrderedDict()
MAX = True
MIN = False
OBJs['min_global_warming'] = ('EX_co2e_mkt', MIN) # minimize total global warming potential

### Flux Constraints ###
FCs = OrderedDict()
FCs['no import'] = {'DM_glyc_mkt':(0,0), 'DM_glc_mkt':(0,0), 'DM_energy_mkt':(0,0)}
FCs['product demand: 3hp = 1000']= {'DM_3hp_mkt':(1000,1000), 'DM_1,3pdo_mkt':(0,0)}
FCs['product demand: pdo = 1000']= {'DM_3hp_mkt':(0, 0), 'DM_1,3pdo_mkt':(1000,1000)}
FCs['product demand: 3hp = 500, pdo=500']= {'DM_3hp_mkt':(500,500), 'DM_1,3pdo_mkt':(500,500)}

FCs['positive profits']= {'PROFIT_res':(0,None), 'PROFIT_agr':(0,None), 'PROFIT_erg':(0,None), 'PROFIT_bch':(0,None)}
FCs['no biodiesel'] = {'energy_from_biodiesel':(0,0)}
FCs['land use < 1'] = {'EX_land_res':(-1,0)}
FCs['land use < 5'] = {'EX_land_res':(-5,0)}
FCs['co2 < 2000'] = {'EX_co2e_mkt':(0, 2000)}


### Ratio Constraints ###

# EU Energy Mix
def add_RC_EU_energy(model):
    add_ratio_constraint(model, 'energy_from_petroleum', 'MX_erg2mkt_energy', 0.25)
    add_ratio_constraint(model, 'energy_from_natural_gas', 'MX_erg2mkt_energy', 0.7)
    add_ratio_constraint(model, 'energy_from_coal', 'MX_erg2mkt_energy', 0.05)

def remove_RC_EU_energy(model):
    remove_ratio_constraint(model, 'ratio_energy_from_petroleum_MX_erg2mkt_energy')
    remove_ratio_constraint(model, 'ratio_energy_from_natural_gas_MX_erg2mkt_energy')
    remove_ratio_constraint(model, 'ratio_energy_from_coal_MX_erg2mkt_energy')

Scenarios = OrderedDict()
base_scenario = {k:v for d in (FCs['no import'], FCs['positive profits']) for k, v in d.iteritems()}
Scenarios['pdo1000, land<1, energy=EU'] = {k:v for d in (base_scenario, FCs['product demand: pdo = 1000'], FCs['land use < 1'], FCs['no biodiesel']) for k, v in d.iteritems()}



In [22]:
### Objective Functions ###
OBJs=OrderedDict()
MAX = True
MIN = False
OBJs['min_global_warming'] = ('EX_co2e_mkt', MIN) # minimize total global warming potential
OBJs['min_eutrofication'] = ('EX_po4e_mkt', MIN) # minimize total eutrophication potential
OBJs['max_bch_profit'] = ('PROFIT_bch', MAX) # maximize profit of biochemical industry
OBJs['min_bch_energy'] = ('MX_bch2mkt_energy', MAX) # minimize energy usage of biochemical industry

S = 'pdo1000, land<1, energy=EU'

if 'energy=EU' in S:
    add_RC_EU_energy(model)  # add EU energy mixing constraint



In [28]:
# sensitivity analysis
solutions = OrderedDict()
for pt in ['pt50','pt100','pt150']:
    for r in model.reactions:
        if 'BPR' in r:
            if pt in r:
                model.bounds[r] = (0, None)
            else:
                model.bounds[r] = (0, 0)
    for O in OBJs:
        print O
        solutions[O, S, pt] = FBA(model, objective={OBJs[O][0]:1}, maximize=OBJs[O][1], constraints = Scenarios[S])



min_global_warming
min_eutrofication
max_bch_profit
min_bch_energy
min_global_warming
min_eutrofication
max_bch_profit
min_bch_energy
min_global_warming
min_eutrofication
max_bch_profit
min_bch_energy


In [32]:
for (O, S, k) in solutions:
    print O, ' -- ', k,': ', solutions[O, S, k].fobj

min_global_warming  --  pt50 :  -2501.86493062
min_eutrofication  --  pt50 :  -9.81268005595
max_bch_profit  --  pt50 :  565.658900232
min_bch_energy  --  pt50 :  -19555.2239571
min_global_warming  --  pt100 :  -1858.88205586
min_eutrofication  --  pt100 :  -9.81268005595
max_bch_profit  --  pt100 :  774.842570386
min_bch_energy  --  pt100 :  -10022.7775533
min_global_warming  --  pt150 :  -1618.45992487
min_eutrofication  --  pt150 :  -9.81268005595
max_bch_profit  --  pt150 :  853.352822845
min_bch_energy  --  pt150 :  -7256.87859956


In [34]:
print model

EX_co2e_mkt: co2e_mkt <-> 
EX_po4e_mkt: po4e_mkt <-> 
EX_waste_organic_mkt: waste_organic_mkt <-> 
EX_waste_cellulosic_mkt: waste_cellulosic_mkt <-> 
EX_waste_inorganic_mkt: waste_inorganic_mkt <-> 
EX_lime_mkt: lime_mkt <-> 
EX_sulfuric_acid_mkt: sulfuric_acid_mkt <-> 
DM_glc_mkt: glc_mkt <-> 0.43 dollar_mkt
DM_glyc_mkt: glyc_mkt <-> 0.18 dollar_mkt
DM_1,3pdo_mkt: 1,3pdo_mkt <-> 1.8 dollar_mkt
DM_3hp_mkt: 3hp_mkt <-> 2.4 dollar_mkt
DM_biomass_mkt: biomass_mkt <-> 
DM_soymeal_mkt: soymeal_mkt <-> 
DM_biodiesel_mkt: biodiesel_mkt <-> 0.9072 dollar_mkt
DM_energy_mkt: energy_mkt <-> 0.02 dollar_mkt
EX_land_res: land_res <-> 
EX_petroleum_res: petroleum_res <-> 
EX_natural_gas_res: natural_gas_res <-> 
EX_coal_res: coal_res <-> 
MX_res2mkt_land: land_res + 41 dollar_mkt <-> land_mkt + 41 dollar_res
PROFIT_res: dollar_res <-> 
MX_res2mkt_petroleum: petroleum_res + 0.765 dollar_mkt <-> petroleum_mkt + 0.765 dollar_res
MX_res2mkt_natural_gas: natural_gas_res + 0.5628 dollar_mkt <-> natural_ga